# Testing for ideas

In [32]:
# samples = os.listdir(DATA_DIR)

# testsample = samples[0]
# image_dir = os.path.join(DATA_DIR, testsample, 'Raw')
# mask_dir = os.path.join(DATA_DIR, testsample, 'Mask')

# nuc_img = mpl.pyplot.imread(os.path.join(image_dir, 'nuclei.tiff'))
# nuc_img = rescale_intensity(nuc_img, (100, 400)) # need to manually set the intensity range
# nuc_img = nuc_img[600:nuc_img.shape[0], 0:600]

# nuc_cvseg = segmentation.chan_vese(nuc_img, mu=0.25, lambda1=1, lambda2=1, tol=1e-3,
#                max_num_iter=200, dt=0.5, init_level_set="checkerboard",
#                extended_output=True)

# fig, axes = mpl.pyplot.subplots(1, 2, figsize=(8, 5))
# ax = axes.flatten()

# ax[0].imshow(nuc_img, cmap="gray")
# ax[0].set_axis_off()
# ax[0].set_title("Original Image", fontsize=12)

# ax[1].imshow(nuc_cvseg[0])
# ax[1].set_axis_off()
# title = f'Chan-Vese segmentation'
# ax[1].set_title(title, fontsize=12)

In [31]:
# nuc_mask = mpl.pyplot.imread(os.path.join(mask_dir, 'mask.tiff'))
# nuc_mask = nuc_mask[600:nuc_mask.shape[0], 0:600]

# fig, axes = mpl.pyplot.subplots(1, 2, figsize=(8, 5))
# ax = axes.flatten()

# ax[0].imshow(nuc_img, cmap="gray")
# ax[0].set_axis_off()
# ax[0].set_title("Original Image", fontsize=12)

# ax[1].imshow(nuc_mask, cmap='gray')
# ax[1].set_axis_off()
# ax[1].set_title('mask', fontsize=12)

## Evolve curve around an object

In [149]:
import numpy as np
from scipy.ndimage import gaussian_filter, laplace
from skimage.io import imshow, imsave, imread
from skimage.draw import disk, circle_perimeter
import matplotlib as mpl
import os
import cv2 as cv
from matplotlib.pyplot import contour

%matplotlib inline

In [2]:
ROOT = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(ROOT, 'Data')

In [217]:
psi = np.zeros((10,10))
psi[0:0,:][np.newaxis,:]

array([], shape=(1, 0, 10), dtype=float64)

In [215]:
# Initialize a guesss of the contour
def initialize(img):
    '''
    Distance transforming image into a family of level sets psi = +/-d
    The psi = 0 level always yields the moving front of a shape, aka the curve.
    '''
    img = np.clip(img, 0, 255)
    img_invert = 255 - img # this step will change boundaries to 0
    img = img.astype('uint8')
    img_invert = img_invert.astype('uint8')
    # conver to single-channgle gray-scale image
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
    zero_level = np.zeros_like(img)
    img_invert = cv.cvtColor(img_invert, cv.COLOR_BGR2GRAY)
    # distance transform the image with a 3 x 3 kernel. 
    dist_img = cv.distanceTransform(img, cv.DIST_L2, 3) 
    img_invert[dist_img==dist_img.max()] = 255 # need to restore the boundaries
    dist_img_invert = cv.distanceTransform(img_invert, cv.DIST_L2, 3) 
    psi = dist_img + dist_img_invert 
    zero_level[psi == psi.min()] = 1
    dist_img = -1 * dist_img # the distances inside the boundaries are negative
    psi = dist_img + dist_img_invert # now the boundaries are fixed and zero
    return psi

def draw_curve(psi, colors='red', levels=[0]):
    '''
    Draw curve U on an image. By default, it is drawn in the red channel
    '''
    # curve at zero-crossing
    # draw levels with equal spacings
    # find the curve that cross zero level
    from matplotlib.pyplot import contour
    X = np.arange(0, psi.shape[1])
    Y = np.arange(0, psi.shape[0])
    X, Y = np.meshgrid(X, Y)
    Z = psi[X, Y]
    return contour(X, Y, Z, levels=levels, colors=colors)

def central_difference(psi):
    '''
    compute central differences along x-axis, y-axis, and between x,y-axes
    '''
    if psi.shape[0] < 2 or psi.shape[1] < 2:
        print('Image has invalid shape: expected sizes of dimensions to be greater than 2.')
    # expanding psi to prevent shrinking in size after taking differences
    psi = np.hstack((psi[:,0][:, np.newaxis], psi))
    psi = np.hstack((psi, psi[:,psi.shape[1]-1][:, np.newaxis]))
    psi = np.vstack((psi[0,:][np.newaxis, :], psi))
    psi = np.vstack((psi, psi[psi.shape[0]-1,:][np.newaxis, :]))
    m, n = psi.shape
    dx = 0.5 * (psi[2:m-1, 1:n-2]-psi[0:m-3, 1:n-2])
    dy = 0.5 * (psi[1:m-2, 2:n-1]-psi[1:m-2, 0:n-3])
    dxx = (psi[2:m-1, 1:n-2] - psi[1:m-2, 1:n-2]) - (psi[1:m-2, 1:n-2] - psi[0:m-3, 1:n-2])
    dyy = (psi[1:m-2, 2:n-1] - psi[1:m-2, 1:n-2]) - (psi[1:m-2, 1:n-2] - psi[1:m-2, 0:n-3])
    dxy = 0.25 * ((psi[2:m-1, 2:n-1] - psi[0:m-3, 2:n-1]) - (psi[2:m-1, 0:n-3] - psi[0:m-3, 0:n-3]))
    return dx, dy, dxx, dyy, dxy

def curve_evolution(img, dt=0.5, max_iter=1000, v=1):
    '''
    evolve the curve based on the magitude the image gradient 
    '''
    psi = initialize(img) # initialize psi
    # imshow(psi)
    dx, dy, dxx, dyy, dxy = central_difference(psi)
    print(dx.shape)
    # dx, dy = np.gradient(psi) # central differences of psi in curvature motion flow
    grad_mag = np.sqrt(dx**2 + dy**2)
    K = (dxx*dy**2 - 2*dy*dx*dxy + dyy*dx**2) / (dx**2 + dy**2)**(1.5)
    # for i in range(max_iter):
    #     dpsi = -1 * K * grad_mag * dt
    #     psi[:] = dt * psi[:] + dpsi
    # img = draw_curve(img, psi)
    # # imshow(img)
    # return img
    

In [216]:
img = imread(os.path.join(DATA_DIR, 'example', 'test.png'))
# U = initialize((img.shape[0],img.shape[1]), r=15)
# dist_transformed_img = distance_transform(img, sigma=2)
curve_evolution(img, max_iter=10)

(49, 49)


C:\Users\Claris_Gu\AppData\Local\Temp\ipykernel_316\3012264359.py:68: RuntimeWarning: invalid value encountered in divide
  K = (dxx*dy**2 - 2*dy*dx*dxy + dyy*dx**2) / (dx**2 + dy**2)**(1.5)
